In [1]:
import zipfile
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors


In [2]:
import zipfile
import pandas as pd
import json

api_token={"username":"luisfhidalgo",
           "key":"d4e4e6425d85ef204447086541cb5b11"}
!rm -rf /roo/.kaggle. # Elimina de forma recursiva y forzada un directorio y su contenido.Por si existe
!mkdir  /root/.kaggle # Crea Directorio

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d zynicide/wine-reviews
archivo_zip = 'wine-reviews.zip'

# Listar los archivos dentro del ZIP
with zipfile.ZipFile(archivo_zip, 'r') as zip_file:
    archivos = zip_file.namelist()
    print("Archivos en el ZIP:")
    for archivo in archivos:
        print(archivo)
#Cargo los Archivos Dataset a los dataframe que voy a utilizar

archivo_zip = 'wine-reviews.zip'

with zipfile.ZipFile(archivo_zip, 'r') as zip_file:

        with zip_file.open('winemag-data-130k-v2.csv') as winemag_150_csv:  # winemag-data-130k-v2.csv
            winemag_df = pd.read_csv(winemag_150_csv)


Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews
License(s): CC-BY-NC-SA-4.0
 98% 50.0M/50.9M [00:02<00:00, 27.3MB/s]
100% 50.9M/50.9M [00:02<00:00, 18.1MB/s]
Archivos en el ZIP:
winemag-data-130k-v2.csv
winemag-data-130k-v2.json
winemag-data_first150k.csv


In [3]:
duplicados = winemag_df.duplicated().sum()
print("Cantidad de duplicados:",duplicados) # son registros exactamente iguales
winemag_df.drop_duplicates(inplace=True)
# Se elimina columna  'Unnamed: 0',
#winemag_df.drop('Unnamed: 0', axis=1, inplace=True)
winemag_df.isnull().sum()
# Se completan valores nulos en 'designation' y 'region_1'
winemag_df.loc[:, 'designation'] = winemag_df['designation'].fillna('Unknown')
winemag_df.loc[:, 'region_1'] = winemag_df['region_1'].fillna('Unknown')
winemag_df.loc[:, 'region_2'] = winemag_df['region_2'].fillna('Unknown')
winemag_df.loc[:, 'taster_twitter_handle'] = winemag_df['taster_twitter_handle'].fillna('Unknown')
winemag_df = winemag_df.dropna(subset=['taster_name'])
# Imputación con la mediana en el campo 'price'
mediana_precio = winemag_df['price'].median()
winemag_df['price'] = winemag_df['price'].fillna(mediana_precio)
# Se elimina  las 5 filas con valores nulos en la columna 'country'
winemag_df.dropna(subset=['country'], inplace=True)


Cantidad de duplicados: 0


<ipython-input-3-0d814e8ec99f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winemag_df['price'] = winemag_df['price'].fillna(mediana_precio)
<ipython-input-3-0d814e8ec99f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winemag_df.dropna(subset=['country'], inplace=True)


In [4]:
winemag_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103664 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             103664 non-null  int64  
 1   country                103664 non-null  object 
 2   description            103664 non-null  object 
 3   designation            103664 non-null  object 
 4   points                 103664 non-null  int64  
 5   price                  103664 non-null  float64
 6   province               103664 non-null  object 
 7   region_1               103664 non-null  object 
 8   region_2               103664 non-null  object 
 9   taster_name            103664 non-null  object 
 10  taster_twitter_handle  103664 non-null  object 
 11  title                  103664 non-null  object 
 12  variety                103664 non-null  object 
 13  winery                 103664 non-null  object 
dtypes: float64(1), int64(2), object(11)
memor

In [5]:
df = winemag_df.head(5000).copy()

In [6]:
### MODELO BASADO EN CONTENIDO ###
def recomendar_vinos_contenido(vino_id, df=df):
    """
    Recomienda vinos similares basados en variedad, país y descripción.
    """
    df["features"] = df["variety"].fillna("") + " " + df["country"].fillna("") + " " + df["description"].fillna("")

    vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)  # Reducir características
    tfidf_matrix = vectorizer.fit_transform(df["features"])

    cosine_sim = cosine_similarity(tfidf_matrix)
    sim_scores = list(enumerate(cosine_sim[vino_id]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]  # Top 5
    wine_indices = [i[0] for i in sim_scores]
    return df.iloc[wine_indices][["title", "variety", "country"]]

In [9]:
# Install the surprise library
#!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505181 sha256=aca300bbc9c143402bcd8ea2ad10559f268c99e28dcbb028e4b9aa69ebcfb27e
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

In [11]:
### MODELO COLABORATIVO ###
def entrenar_modelo_colaborativo(df):
    """
    Entrena un modelo KNN más ligero basado en puntuaciones de usuarios.
    """
    reader = Reader(rating_scale=(df["points"].min(), df["points"].max()))
    data = Dataset.load_from_df(df[["taster_name", "title", "points"]], reader)
    trainset, testset = train_test_split(data, test_size=0.2)

    model = KNNBasic()  # Algoritmo más eficiente
    model.fit(trainset)

    predictions = model.test(testset)
    print(f"RMSE: {accuracy.rmse(predictions)}")
    return model

def recomendar_vinos_usuario(usuario, model, df, n=5):
    """
    Recomienda vinos a un usuario basado en puntuaciones previas.
    """
    vinos_no_probados = df["title"].unique()
    predicciones = [model.predict(usuario, vino) for vino in vinos_no_probados[:100]]  # Limitar cantidad
    predicciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:n]
    return [pred.iid for pred in predicciones]

# Entrenar modelo colaborativo
modelo_colab = entrenar_modelo_colaborativo(df)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 2.8841
RMSE: 2.8841491807810495


In [12]:
# Ejemplo de recomendaciones
print("Recomendaciones Basadas en Contenido:")
print(recomendar_vinos_contenido(0))

print("\nRecomendaciones Basadas en Colaboración:")
print(recomendar_vinos_usuario("John Doe", modelo_colab, df))


Recomendaciones Basadas en Contenido:
                                                  title     variety country
2000  Feudi del Pisciotto 2013 Baglio del Sole Inzol...     Inzolia   Italy
5991  Barone Sergio 2015 Alègre Grillo (Terre Sicili...      Grillo   Italy
908                   Cascina Bruciata 2013  Barbaresco    Nebbiolo   Italy
1181            Bel Colle 2013 Montersino  (Barbaresco)    Nebbiolo   Italy
1036                Vivera 2010 Salisire Bianco  (Etna)  Carricante   Italy

Recomendaciones Basadas en Colaboración:
['Nicosia 2013 Vulkà Bianco  (Etna)', 'Quinta dos Avidagos 2011 Avidagos Red (Douro)', 'Rainstorm 2013 Pinot Gris (Willamette Valley)', 'St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore)', "Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)"]
